<table style="width: 100%">
    <tr style="background: #ffffff">
        <td style="padding-top:25px; width: 180px">
            <img src="https://mci.edu/templates/mci/images/logo.svg" alt="Logo">
        </td>
        <td style="width: 100%">
            <div style="width: 100%; text-align:right"><font style="font-size:38px"><b>Softwaredesign</b></font></div>
            <div style="padding-top:0px; width: 100%; text-align:right"><font size="4"><b>WS 2023</b></font></div>
        </td>
    </tr>
</table>

---

# Netzwerkanalyse einer elektrischen Schaltung mit dem Maschenstromverfahren

Mit Hilfe des Maschenstromverfahrens können die Ströme in einem elektrischen Netzwerk berechnet werden. Das Verfahren basiert auf der Anwendung des Kirchhoff'schen Maschengesetzes auf die einzelnen Maschen des Netzwerks. 

Im allgemeinen wird das Maschenstromverfahren in folgenden Schritten angewendet:
1. Aufstellen der Maschengleichungen
2. Aufstellen der Knotengleichungen
3. Bilden eines Gleichungssystems
4. Identifizieren von etwaigen linear abhängigen Gleichungen
5. Linear abhägnige Gleichungen eliminieren
6. Lösen des Gleichungssystems

Das Gleichungssystem hat hierbei allgemein die Form:

$\boldsymbol{Z} \, \vec{I} = \vec{U}$

wobei $\boldsymbol{Z}$ für die Matrix an Maschenimpedanzen, $\vec{I}$ für den Spaltenvektor der Maschenströme und $\vec{U}$ für den Spaltenvektor der Maschenspannungen steht.

Bei einer reinen Betrachtung unter Gleichspannung vereinfacht sich die Matrix $\boldsymbol{Z}$ zur Matrix der frequenzunabhängigen Maschenwiderstände $\boldsymbol{R}$ und das Gleichungssystem hat die Form:

$\boldsymbol{R} \, \vec{I} = \vec{U}$

Das Lösen solch eines Gleichungssystems kann händisch erfolgen, ist jedoch bei komplexeren Schaltungen sehr aufwendig. Daher soll im Folgenden ein Programm entwickelt werden, welches die Berechnung der Maschenströme automatisiert.

Da ein lineares Gleichungssystem gelöst werden soll, bietet sich natürlich die Verwendung der Bibliothek `numpy` an.

In [15]:
import numpy as np

### 1. & 2. Aufstellen der Maschen- & Knotengleichungen

Für die Schaltung in folgender Abbildung können laut den Kirchhoff'schen Regeln folgende Maschen- und Knotengleichungen aufgestellt werden:

<img style="background-color:white; padding:10px; width:40%" src="mesh_analysis_circuit.png"></img>

$\text{M1}: U_{R_2} + U_{R_5} + U_5 - U_{R_4} = 0$

$\text{M2}: U_{R_1} - U_{R_3} - U_5 - U_{R_5} = 0$

$\text{M3}: U_{R_4} + U_{R_3} - U_{R_0} - U_0 = 0$

$\text{K1}:  I_2 - I_1 - I_5 = 0$

$\text{K2}:  I_4 + I_5 - I_3 = 0$

$\text{K3}:  I_0 + I_1 + I_3 = 0$

$\text{K4}: -I_0 - I_2 - I_4 = 0$

Nun müssen noch die bekannten Größen der Schaltung $R_0$ bis $R_5$, $U_0$ und $U_5$ definiert werden.

In [16]:
#Parameter
R0 = 100 #in Ohm
R1 = 120 #in Ohm
R2 = 220 #in Ohm
R3 = 330 #in Ohm
R4 = 470 #in Ohm
R5 = 560 #in Ohm
U0 =  5  #in V
U5 = 10  #in V

### 3. Bilden eines Gleichungssystems

Aus den Maschen- und Knotengleichungen und den bekannten Parametern kann nun das Gleichugnssystem $\boldsymbol{R} \, \vec{I} = \vec{U}$ aufgestellt werden.

Hierbei muss beachtet werden, dass die Maschenströme $I_0$ bis $I_5$ und die Maschenspannungen $U_0$ bis $U_5$ in der gleichen Reihenfolge in den Vektoren $\vec{I}$ und $\vec{U}$ auftreten, wie sie in den Maschen- und Knotengleichungen aufgeführt sind.
Ebenso gilt hier natürlich der Zusammenhang $U_{R_i} = R_i \, I_i$.

In [17]:
R = np.array([[0,-1,1,0,0,-1],
              [0,0,0,-1,1,1],
              [1,1,0,1,0,0],
              [-1,0,-1,0,-1,0],
              [0,0,R2,0,-R4,R5],
              [0,R1,0,-R3,0,-R5],
              [-R0,0,0,R3,R4,0]])

U = np.array([0,0,0,U5,U5,U0])

print(R)
print(R.shape)


[[   0   -1    1    0    0   -1]
 [   0    0    0   -1    1    1]
 [   1    1    0    1    0    0]
 [  -1    0   -1    0   -1    0]
 [   0    0  220    0 -470  560]
 [   0  120    0 -330    0 -560]
 [-100    0    0  330  470    0]]
(7, 6)


### 4. & 5. Identifizieren und eliminieren von linear abhängigen Gleichungen

In unserer Schaltung liegen 6 unbekannte Ströme vor ($I_0$ bis $I_5$) daher muss unser Gleichungssystem 6 Gleichungen enthalten. Da wir jedoch 7 Gleichungen aufgestellt haben, muss eine der Gleichungen linear abhängig sein.
Dies kann auch aus der Anzahl an Zeilen und Spalten der Matrix $\boldsymbol{R}$ (`R.shape`) bzw. an deren Rang (`np.linalg.matrix_rank(R)`) erkannt werden.

Bei genauerem Betrachten der Knotengleichungn fällt auf, dass die Knotengleichung K4 die negative Summe der Knotengleichungen K1, K2 und K3 ist. Daher kann die Gleichung K4 eliminiert werden. Im `numpy` array `R` und im Vektor `U` kann dies mit der Funktion `np.delete(...)` erfolgen.

In [18]:
R = np.delete(R,3,0)
print(R.shape)
print(R)

(6, 6)
[[   0   -1    1    0    0   -1]
 [   0    0    0   -1    1    1]
 [   1    1    0    1    0    0]
 [   0    0  220    0 -470  560]
 [   0  120    0 -330    0 -560]
 [-100    0    0  330  470    0]]


### 6. Lösen des Gleichungssystems

Das Gleichungssystem kann nun mit Hilfe der Funktion `numpy.linalg.solve()` gelöst werden.

In [19]:
I = np.linalg.solve(R,U)
print(I)
print(I.shape)

[-0.05636415  0.05921094  0.05571942 -0.00284678  0.00064473 -0.00349152]
(6,)


Das Ergbnis für die Ströme sollte in diesem Fall der Vektor `[-0.015282    0.0181537   0.00587889 -0.0028717   0.00940311 -0.01227481]` sein.

Dies setzt voraus, dass die Reihenfolge der Ströme & Spannungen konsistent mit den Maschen- und Knotengleichungen gewählt wurde.

In [20]:
#Für überbestimmte Gleichungssysteme kann auch eine Lösung im Sinne der kleinsten Quadrate berechnet werden

#Hier müssen die originalen Werte für R und U eingesetzt werden
# R_orig und U_orig sind die Matritzen/Vektoren bevor die linear abhängige Zeile entfernt wurde
I_lsq = np.linalg.lstsq(R, U, rcond=None)[0]
print(I_lsq)

#Diese Lösung ist aber für den aktuellen Fall nicht besonders genau!
relative_error = 1 - (I / I_lsq)
print(F"Relative error of the LSQ solution:\n{relative_error}")

[-0.05636415  0.05921094  0.05571942 -0.00284678  0.00064473 -0.00349152]
Relative error of the LSQ solution:
[-4.46309656e-14  2.19824159e-14  7.54951657e-15 -1.61648472e-13
  3.43058915e-13 -9.10382880e-15]


Hier sollte ein relativ großer Fehler von `[-0.47036898  0.02452364  0.25125603  0.30772249  0.17076191 -0.07440576]` für die jeweiligen Strömme auftreten, da sich überbestimmte Gleichungssystem nicht exakt lösen lassen.

Die hier gewählte Variante der Näherungslösung ist in diesem Fall suboptimal.

Weiters können nun auch die Spannugnsabfälle über die Widerstände $R_0$ bis $R_5$ berechnet werden.

Hierzu wird eine neue Diagonalmatrix aufgestellt, welche die Widerstände $R_0$ bis $R_5$ auf der Hauptdiagonalen enthält. Diese Matrix wird mit dem Vektor der Maschenströme multipliziert und es ergibt sich der Vektor der Spannungsabfälle $\vec{U}_R$.

$
\begin{bmatrix}
    R_0 & 0 & 0 & 0 & 0 & 0 \\
    0 & R_1 & 0 & 0 & 0 & 0 \\
    0 & 0 & R_2 & 0 & 0 & 0 \\
    0 & 0 & 0 & R_3 & 0 & 0 \\
    0 & 0 & 0 & 0 & R_4 & 0 \\
    0 & 0 & 0 & 0 & 0 & R_5 \\
\end{bmatrix}
\vec{I} = \vec{U}_R
$

Um sich hier möglichst viel Tipparbeit zu sparen kann die Funktion `np.diag(...)` verwendet werden.

In [21]:

R_original = np.array([R0,R1,R2,R3,R4,R5])
R_diag = np.diag(R_original)
U_R = np.dot(R_diag,I)
print(R_diag)
print(U_R)

[[100   0   0   0   0   0]
 [  0 120   0   0   0   0]
 [  0   0 220   0   0   0]
 [  0   0   0 330   0   0]
 [  0   0   0   0 470   0]
 [  0   0   0   0   0 560]]
[-5.63641517  7.10531234 12.25827249 -0.93943887  0.3030237  -1.9552488 ]


### Für Interessierte: LTSpice-Simulation der Schaltung aus Python heraus

Falls Sie an weiteren Anwendungen von Python in Kombination mit anderen Softwarepaketen interessiert sind, können Sie sich gerne mit den folgenden Themen beschäftigen.

Um komplexere Schaltungen zu simulieren werden üblicherweise Simulationsprogramme wie beispielsweise [LTSpice](https://de.wikipedia.org/wiki/LTspice) verwendet.
Python bietet aber auch die Möglichkeit über das `PyLTSpice` package LTSpice-Simulationen zu starten, und die Ergebnisse auszuwerten. Hier können auch zuvor im Python-Code Werte und Paramter gesetzt werden, welche dann in der Simulation verwendet werden.

Hierfür muss lokal sowohl [LTSpice](https://www.analog.com/en/design-center/design-tools-and-calculators/ltspice-simulator.html) als auch das `PyLTSpice` package installiert werden. Die Installation von `PyLTSpice` erfolgt wie gewohnt über den Befehl `pip install PyLTSpice`.

In [22]:
from PyLTSpice import SimRunner, SpiceEditor, RawRead

Es kann ein Template für die LTSpice-Simulation erstellt werden (`mesh_analysis_ltspice.asc`), dessen Parameter dann in Python gesetzt bzw. verändert werden können.

In [23]:
runner = SimRunner(output_folder='./')

netlist = SpiceEditor("mesh_analysis_ltspice.asc") #Create the .net list for the simulation from the *.asc file
netlist_name = netlist.netlist_file.name

netlist.set_component_value('R0', str(R0))
netlist.set_component_value('R1', str(R1))
netlist.set_component_value('R2', str(R2))
netlist.set_component_value('R3', str(R3))
netlist.set_component_value('R4', str(R4))
netlist.set_component_value('R5', str(R5))

#voltages are labeled with V in LTSpice
netlist.set_component_value('V0', str(U0))
netlist.set_component_value('V5', str(U5))

netlist.add_instructions(".op") #DC operating point analysis

FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden

Sobald die Schaltungsbeschreibung in Form der `netlist` erstellt wurde, kann diese mit Hilfe der Funktion `run_now(...)` ausgeführt werden.

In [ ]:
raw_file, log_file = runner.run_now(netlist, run_filename=netlist_name)
runner.wait_completion()

NameError: name 'netlist' is not defined

Sobald die Simulation abgeschlossen ist, können die Ergebnisse aus der `*.raw`-Datei ausgelesen werden. Im Fall der "DC operating point analysis" werden die Spannungen und Ströme für den definierten DC-Fall betrachtet. Diese können über ihren Namen mit Hilfe der Funktion `get_trace(...)` ausgelesen werden.

In [ ]:
raw = RawRead(raw_file)

#extract results with the specified names
names = ['I(R0)', 'I(R1)', 'I(R2)', 'I(R3)', 'I(R4)', 'I(R5)']
currents = []
for i, name in enumerate(names):
    currents.append(raw.get_trace(name))

I_ltspice = np.array(currents).ravel()
print(I_ltspice)

Um zu vergleichen wie genau die Ergebnisse von LTSpice und unserer Lösung mittels Maschenstromverfahren wirklich sind, wird der relative Fehler zwischen beiden Größen bestimmt. 

Hierfür muss jedoch darauf geachtet werden, dass für das Maschenstromverfahren die Vorzeichen der Ströme von der eingezeichneten Richtung abhängen. Bei den Ergebnissen in LTSpice hängen diese vom definierten Massepotenzial ab, und sind damit nicht zwangsläufig ident zu jenen des Maschenstromverfahrens

In [ ]:
relative_error_ltspice = 1 - (abs(I) / abs(I_ltspice))
print(F"Relative error of the LTSpice simulation:\n{relative_error_ltspice}")